In [1]:
# Import necessary packages
import os
import rasterio as rio
import earthpy as et

In [2]:
# Get data and set working directory
et.data.get_data("colorado-flood")
os.chdir(os.path.join(et.io.HOME, 'earth-analytics'))

Extracted output to /home/pleroy/earth-analytics/data/colorado-flood/.


In [3]:
# Define relative path to file
lidar_dem_path = os.path.join("data", "colorado-flood", "spatial", 
                              "boulder-leehill-rd", "pre-flood", "lidar",
                              "pre_DTM.tif")

with rio.open(lidar_dem_path) as lidar_dem:
    lidar_dem.bounds

In [4]:
# View generate metadata associated with the raster file
lidar_dem.meta

{'driver': 'GTiff',
 'dtype': 'float32',
 'nodata': -3.4028234663852886e+38,
 'width': 4000,
 'height': 2000,
 'count': 1,
 'crs': CRS.from_epsg(32613),
 'transform': Affine(1.0, 0.0, 472000.0,
        0.0, -1.0, 4436000.0)}

In [5]:
# What is the spatial resolution?
lidar_dem.res

(1.0, 1.0)

In [6]:
# View image structure
with rio.open(lidar_dem_path) as lidar_dem:
    print(lidar_dem.tags(ns='IMAGE_STRUCTURE'))
    lidar_dem_mask = lidar_dem.dataset_mask()

{'COMPRESSION': 'LZW', 'INTERLEAVE': 'BAND'}


In [7]:
# View data mask
lidar_dem_mask

array([[  0,   0,   0, ..., 255, 255, 255],
       [  0,   0,   0, ..., 255, 255, 255],
       [  0,   0,   0, ..., 255, 255, 255],
       ...,
       [  0,   0,   0, ..., 255, 255, 255],
       [  0,   0,   0, ..., 255, 255, 255],
       [  0,   0,   0, ..., 255, 255, 255]], dtype=uint8)

In [8]:
print(lidar_dem.crs)
print(lidar_dem.bounds)

EPSG:32613
BoundingBox(left=472000.0, bottom=4434000.0, right=476000.0, top=4436000.0)


In [10]:
dataset = rio.open("/home/pleroy/TMP/2019_07_12_14_12_21_img.tif")

In [12]:
dataset.name, dataset.mode, dataset.closed

('/home/pleroy/TMP/2019_07_12_14_12_21_img.tif', 'r', False)

In [13]:
dataset.count, dataset.width, dataset.height

(4, 3000, 500)

In [14]:
{i: dtype for i, dtype in zip(dataset.indexes, dataset.dtypes)}

{1: 'uint8', 2: 'uint8', 3: 'uint8', 4: 'uint8'}

In [15]:
dataset.indexes, dataset.dtypes

((1, 2, 3, 4), ('uint8', 'uint8', 'uint8', 'uint8'))

In [16]:
dataset.bounds, dataset.transform

(BoundingBox(left=1326195.107668397, bottom=7218335.049419026, right=1327708.3130845958, top=7218683.26820938),
 Affine(0.4989150689622367, -0.032920418624412966, 1326211.5678777092,
        -0.032920418624412966, -0.4989150689622367, 7218683.26820938))

In [17]:
upperleft = dataset.transform * (0, 0)

In [18]:
lowerright = dataset.transform * (dataset.width, dataset.height)

In [19]:
dataset.crs

CRS.from_epsg(3948)

In [31]:
dataset.crs.to_wkt()

'PROJCS["RGF93 / CC48",GEOGCS["RGF93",DATUM["Reseau_Geodesique_Francais_1993",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0],AUTHORITY["EPSG","6171"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4171"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",48],PARAMETER["central_meridian",3],PARAMETER["standard_parallel_1",47.25],PARAMETER["standard_parallel_2",48.75],PARAMETER["false_easting",1700000],PARAMETER["false_northing",7200000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","3948"]]'

# Creating data

In [20]:
import numpy as np
x = np.linspace(-4.0, 4.0, 240)
y = np.linspace(-3.0, 3.0, 180)
X, Y = np.meshgrid(x, y)
Z1 = np.exp(-2 * np.log(2) * ((X - 0.5) ** 2 + (Y - 0.5) ** 2) / 1 ** 2)
Z2 = np.exp(-3 * np.log(2) * ((X + 0.5) ** 2 + (Y + 0.5) ** 2) / 2.5 ** 2)
Z = 10.0 * (Z2 - Z1)

In [21]:
from rasterio.transform import Affine
res = (x[-1] - x[0]) / 240.0
transform = Affine.translation(x[0] - res / 2, y[0] - res / 2) * Affine.scale(res, res)

In [22]:
transform

Affine(0.03333333333333333, 0.0, -4.016666666666667,
       0.0, 0.03333333333333333, -3.0166666666666666)

In [28]:
new_dataset = rio.open(
    '/home/pleroy/TMP/new.tif',
    'w',
    driver='GTiff',
    height=Z.shape[0],
    width=Z.shape[1],
    count=1,
    dtype=Z.dtype,
    crs='EPSG:3948', # crs='+proj=latlong',
    transform=transform,)

In [29]:
new_dataset.write(Z, 1)

In [30]:
new_dataset.close()

In [32]:
new_dataset.crs.to_wkt()

'PROJCS["RGF93 / CC48",GEOGCS["RGF93",DATUM["Reseau_Geodesique_Francais_1993",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6171"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4171"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",48],PARAMETER["central_meridian",3],PARAMETER["standard_parallel_1",47.25],PARAMETER["standard_parallel_2",48.75],PARAMETER["false_easting",1700000],PARAMETER["false_northing",7200000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","3948"]]'

In [33]:
dataset.crs.to_wkt()

'PROJCS["RGF93 / CC48",GEOGCS["RGF93",DATUM["Reseau_Geodesique_Francais_1993",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0],AUTHORITY["EPSG","6171"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4171"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",48],PARAMETER["central_meridian",3],PARAMETER["standard_parallel_1",47.25],PARAMETER["standard_parallel_2",48.75],PARAMETER["false_easting",1700000],PARAMETER["false_northing",7200000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","3948"]]'